In [2]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

max_features = 10000 # 특성으로 사용할 단어의 수
maxlen = 20  # 사용할 텍스트의 길이(가장 빈번한 20 개의 단어만 사용합니다)

(x_train, y_train), (x_test, y_test) =imdb.load_data(num_words=max_features)

# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) # 가장 빈번한 단어 최대 maxlen(20)개만
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
print(x_train.shape)

(25000, 20)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding

model = Sequential()

# 가능한 토큰의 개수(1,0000)와 임베딩 차원(8)로 설정 
# 꼭 8일 필요는 없고, 데이터의 양에 따라 적절히 설정
# 최대 유니크한 개수 10000개
# 몇차원으로 임베딩할거야? -> 8
# 한 문장당 몇개 단어임? -> 20(==maxlen)
model.add(Embedding(10000, 8, input_length=maxlen))   
    
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됨
# 꼭 8로 할 필요는 없고, 데이터의 크기에 따라 적절히 설정하면 됨
model.add(Flatten()) #(samples, maxlen * 8) 크기의 2D 텐서로 펼침

# 분류기를 추가
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train,y_train,
                    verbose=0,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 8)             80000     
                                                                 
 flatten_1 (Flatten)         (None, 160)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 161       
                                                                 
Total params: 80161 (313.13 KB)
Trainable params: 80161 (313.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


맨 처음 데이터는 25000 개의 리뷰글. 각각의 리뷰글에는 20개의 단어가 있음.<br/>
한 글씩 임베딩 과정을 거치면 8개의 숫자가 된다.<br/>
(25000,20)의 2차원 데이터가 (25000, 20, 8)의 3차원 데이터가 된다.<br/>
<br/>
하지만, Dense는 2차원 데이터를 매개변수로 하기 때문에 Flatten()을 통해 2차원으로 바꿔주는 과정이 필요하다.

리뷰에서 20개의 단어만 사용한 것 치고는 70%이상의 좋은 결과. <br/>
하지만,각 단어를 독립적으로 다루었음. 단어 사이의 관계나 문장의 구조를 고려하지 않았음. <br/>
각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환 층이나 1D 합성곱 층을 추가하는 것이 좋음. 